### `Calculate LD statistics and FDR-adjusted p values for ANOVA and t tests`

In [2]:
import numpy as np
import pandas as pd
import scipy.cluster.hierarchy as shc
import scipy.stats as stats
from statsmodels.stats.multitest import multipletests
import matplotlib.pyplot as plt
import seaborn as sb 

In [38]:
#prunes non-QTL markers, extends multiallelic SNPs, gets original VCF ID, generated unique MID
df5 = pd.read_csv('../../WFc2.vcf', sep = '\t')
df4 = pd.read_table('../Sequence and SNP data/Bout340can.tsv', header = None)
df4['Start'] = df4[[8,9]].min(axis = 1)
df4['End'] = df4[[8,9]].max(axis = 1)
t = []
for s,e in df4[[8,9]].itertuples(index = False):
    if s < e:
        t.append('forward')
    else:
        t.append('reverse')
df4['Strand'] = t

t = []
for c, p in df5[['#CHROM', 'POS']].itertuples(index = False):
    f = False
    for l,o,s,e in df4[df4[1] == c][[0, 'Strand', 'Start', 'End']].itertuples(index = False):
        if s <= p <= e:
            t.append((l, o, s, e))
            f = True
            break
    if f == False:
        t.append(('None', o, s, e))
        
df5['Gene'] = list(zip(*t))[0]
df5['Strand'] = list(zip(*t))[1]
df5['Start'] = list(zip(*t))[2]
df5['End'] = list(zip(*t))[3]

df5 = df5[df5['Gene'] != 'None']

t = []
for i,r in df5.iterrows():
    if ',' in r['ALT']:
        for a in r['ALT'].split(','):
            r2 = r.copy()
            r2.loc['ALT'] = a
            t.append(r2)
    else:
        t.append(r)
        
df6 = pd.DataFrame(t)
df6 = df6.applymap(lambda x: 0 if x == '0/0' else x).applymap(lambda x: str(x).count('/0') if '/0' in str(x) else x).applymap(lambda x: str(x).count('0/') if '0/' in str(x) else x).applymap(lambda x: 2 if '/' in str(x) else x)

In [41]:
df6.drop(columns = ['Gene', 'Strand', 'Start', 'End'])#.to_csv('../../WFC2L.vcf', sep = '\t', index = False)

In [ ]:
#Benjamini-Hochberg FDR correction

In [11]:
df = pd.read_csv('../../WFC2L.vcf', sep = '\t')

In [5]:
df2 = pd.read_csv('../One-way ANOVA.csv')

In [8]:
df3 = pd.read_csv('../t-test.csv')

In [9]:
df2['FDR adjusted p value'] = multipletests(df2['p value'], alpha=0.01, method='fdr_bh', returnsorted=True)[1]
df3['FDR adjusted p value'] = multipletests(df3['p value'], alpha=0.01, method='fdr_bh', returnsorted=True)[1]

In [10]:
#df2.to_csv('One-way ANOVA for qq.csv', index = False)
#df3.to_csv('t-test for qq.csv', index = False)

In [12]:
df4 = df2[df2['FDR adjusted p value'] < 0.01]

In [13]:
df5 = df3[df3['FDR adjusted p value'] < 0.01]

In [14]:
#df4.to_csv('../Figures and tables/S9 Table One-way ANOVA results.csv', index = False)
#df5.to_csv('../Figures and tables/S10 Table Students t test results.csv', index = False )

In [ ]:
#LD statistics

In [12]:
df6 = pd.read_excel('../Figures and tables/Table 4 Frameshift and nonsense mutants.xlsx')

In [13]:
df7 = pd.read_csv('../Figures and tables/S12 Table All SNP substitution effects.csv')[['ID', 'Effect', 'MID', 'QTL']]

In [14]:
df7 = df7[df7['Effect'] == 'synonymous']

In [65]:
#marker LD and mean QTL LD 
l = []
dists = []
for eyed in df6['MID'].to_list():
    q = df6[df6['MID'] == eyed]['QTL'].to_list()[0]
    tl = df7[(df7['QTL'] == q)]['MID'].to_list()
    tl.append(eyed)
    tdf = df.loc[tl, :]
    mean = (tdf.iloc[:,9:].transpose().corr() **2).mean(axis = 1).mean()
    marker = (tdf.iloc[:,9:].transpose().corr() **2)[tdf[tdf['MID'] == eyed].index[0]].mean()

    l.append((eyed, marker, mean))

In [104]:
q = df6[df6['MID'] == eyed]['QTL'].to_list()[0]

In [66]:
pd.DataFrame(l).to_clipboard()